In [ ]:
import numpy as np
import awkward

In [ ]:
import logging
logging.basicConfig(level=logging.INFO, format='[%(asctime)s] %(levelname)s: %(message)s')

In [ ]:
def stack_arrays(a, keys, axis=-1):
    flat_arr = np.stack([a[k].flatten() for k in keys], axis=axis)
    return awkward.JaggedArray.fromcounts(a[keys[0]].counts, flat_arr)

In [ ]:
def pad_array(a, maxlen, value=0., dtype='float32'):
    x = (np.ones((len(a), maxlen)) * value).astype(dtype)
    for idx, s in enumerate(a):
        if not len(s):
            continue
        trunc = s[:maxlen].astype(dtype)
        x[idx, :len(trunc)] = trunc
    return x

In [ ]:
class Dataset(object):

    def __init__(self, filepath, feature_dict = {}, label='label', pad_len=100, data_format='channel_first'):
        self.filepath = filepath
        self.feature_dict = feature_dict
        if len(feature_dict)==0:
            feature_dict['points'] = ['part_etarel', 'part_phirel']
            feature_dict['features'] = ['part_pt_log', 'part_e_log', 'part_etarel', 'part_phirel']
            feature_dict['mask'] = ['part_pt_log']
        self.label = label
        self.pad_len = pad_len
        assert data_format in ('channel_first', 'channel_last')
        self.stack_axis = 1 if data_format=='channel_first' else -1
        self._values = {}
        self._label = None
        self._load()

    def _load(self):
        logging.info('Start loading file %s' % self.filepath)
        counts = None
        with awkward.load(self.filepath) as a:
            self._label = a[self.label]
            for k in self.feature_dict:
                cols = self.feature_dict[k]
                if not isinstance(cols, (list, tuple)):
                    cols = [cols]
                arrs = []
                for col in cols:
                    if counts is None:
                        counts = a[col].counts
                    else:
                        assert np.array_equal(counts, a[col].counts)
                    arrs.append(pad_array(a[col], self.pad_len))
                self._values[k] = np.stack(arrs, axis=self.stack_axis)
        logging.info('Finished loading file %s' % self.filepath)


    def __len__(self):
        return len(self._label)

    def __getitem__(self, key):
        if key==self.label:
            return self._label
        else:
            return self._values[key]
    
    @property
    def X(self):
        return self._values
    
    @property
    def y(self):
        return self._label

    def shuffle(self, seed=None):
        if seed is not None:
            np.random.seed(seed)
        shuffle_indices = np.arange(self.__len__())
        np.random.shuffle(shuffle_indices)
        for k in self._values:
            self._values[k] = self._values[k][shuffle_indices]
        self._label = self._label[shuffle_indices]

In [ ]:
username='lbenato'#type here your username!
srcDir = '/nfs/dust/cms/user/'+username+'/ML_LLP/TopTaggingDataset/'

train_dataset = Dataset(srcDir+'converted/train_file_0.awkd', data_format='channel_last')
val_dataset = Dataset(srcDir+'converted/val_file_0.awkd', data_format='channel_last')
test_dataset = Dataset(srcDir+'converted/test_file_0.awkd', data_format='channel_last')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tf_keras_model import get_particle_net, get_particle_net_lite

In [ ]:
model_type = 'particle_net_lite' # choose between 'particle_net' and 'particle_net_lite'
num_classes = train_dataset.y.shape[1]
input_shapes = {k:train_dataset[k].shape[1:] for k in train_dataset.X}
if 'lite' in model_type:
    model = get_particle_net_lite(num_classes, input_shapes)
else:
    model = get_particle_net(num_classes, input_shapes)

In [ ]:
# Training parameters
batch_size = 1024 if 'lite' in model_type else 384
epochs = 30

In [ ]:
def lr_schedule(epoch):
    lr = 1e-3
    if epoch > 10:
        lr *= 0.1
    elif epoch > 20:
        lr *= 0.01
    logging.info('Learning rate: %f'%lr)
    return lr

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()

In [ ]:
# Prepare model model saving directory.
import os
save_dir = srcDir+'model_checkpoints'
model_name = '%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = keras.callbacks.ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = keras.callbacks.LearningRateScheduler(lr_schedule)
progress_bar = keras.callbacks.ProgbarLogger()
callbacks = [checkpoint, lr_scheduler, progress_bar]

In [ ]:
train_dataset.shuffle()
model.fit(train_dataset.X, train_dataset.y,
          batch_size=batch_size,
#           epochs=epochs,
          epochs=1, # --- train only for 1 epoch here for demonstration ---
          validation_data=(val_dataset.X, val_dataset.y),
          shuffle=True,
          callbacks=callbacks)

# Save your model at the end of the training
model_label = '1'#you can change this label
output_file = 'model_'+model_label+'.h5'
model.save(save_dir+output_file)
print("Model saved in ", save_dir+output_file)

In [ ]:
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

# Load your model, in case you don't want to retrain
model_label = '1'
output_file = 'model_'+model_label+'.h5'
print("Loading model... ", save_dir+output_file)
model = keras.models.load_model(save_dir+output_file)

# Compute probabilities of being signal/background, given the input features X
probs = model.predict(test_dataset.X)
print(probs)

In [ ]:
# Calculates area under ROC
AUC = roc_auc_score(test_dataset.y, probs)
print("Test Area under Curve = {0}".format(AUC))

# You can play a bit with these printouts to understand what is the shape of the objects you arer dealing with
print( (test_dataset.y[0]).shape)
#print(probs)
print(probs[:,1])
#print(test_dataset.y)
print(test_dataset.y[:,1])

# We now want to plot a histogram of the probability of being signal, 
# for a true signal sub-sample, and the probability of being signal, 
# for a true background sample.
sign_array = np.multiply(test_dataset.y[:,1],probs[:,1])
back_array = np.multiply(test_dataset.y[:,1],probs[:,0])
# Remove zeros from arrays
sign = sign_array[sign_array != 0]
back = back_array[back_array != 0]

plt.figure(figsize=(8,5))
plt.rcParams.update({'font.size': 15}) #Larger font size
plt.hist(back, 50, color='blue', edgecolor='blue', lw=2, label='background', alpha=0.3)
plt.hist(sign, 50, color='red', edgecolor='red', lw=2, label='signal', alpha=0.3)
plt.xlim([-0.05, 1.05])
plt.xlabel('Event probability of being classified as signal')
plt.legend(loc="upper center")
plt.yscale('log')
plt.grid(True)

out_prob = 'probab_'+'model_'+model_label
#plt.savefig(out_prob+'.png')
#plt.savefig(out_prob+'.pdf')
plt.show()

# Extract true positive rate and false positive rate
fpr, tpr, _ = roc_curve(test_dataset.y[:,1], probs[:,1])
plt.figure(figsize=(8,7))
plt.rcParams.update({'font.size': 15}) #Larger font size
plt.plot(fpr, tpr, color='crimson', lw=2, label='ROC curve (area = {0:.4f})'.format(AUC))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([-0.05, 1.05])
plt.ylim([0.0, 1.05])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(loc="lower right")
plt.grid(True)

out = 'ROC_'+'model_'+model_label
#plt.savefig(save_dir+out+.png')
#plt.savefig(save_dir+out+.pdf')
plt.show()